## LSSTCam Astrometry learning with GAIA

Craig Lage - 19-May-25

In [ ]:
import numpy as np
from scipy.ndimage import rotate
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
import lsst.afw.geom as afwGeom
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from astroquery.gaia import Gaia
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.lsst import LsstCam
from lsst.obs.base import createInitialSkyWcsFromBoresight, createInitialSkyWcs
from astropy.coordinates import EarthLocation
location = EarthLocation.of_site('Rubin:Simonyi')
import lsst.geom as geom
from lsst.summit.utils.plotting import plot
from os import listdir
import healpy as hp
from astropy.table import Table, vstack
from astropy.coordinates import angular_separation
import cv2
import sys

In [ ]:
def MakeShiftedWCS(camera, ref_detector, ref_wcs, img_detector):
    img_bbox = img_detector.getBBox()
    ref_bbox = ref_detector.getBBox()
    ref_detName = ref_detector.getName()
    img_detName = img_detector.getName()
    ref_lct = LsstCameraTransforms(camera, ref_detName)
    refX, refY = ref_lct.ccdPixelToFocalMm(ref_bbox.centerX, ref_bbox.centerY, ref_detName)
    img_lct = LsstCameraTransforms(camera, img_detName)
    imgX, imgY = img_lct.ccdPixelToFocalMm(img_bbox.centerX, img_bbox.centerY, img_detName)
    # The factor of 100 below is 1000 microns/mm / 10 microns/pixel = 100 pixels/mm.
    fpX = (imgX - refX) * 100 + img_bbox.centerX # Convert to pixels and recenter
    fpY = (imgY - refY) * 100 + img_bbox.centerY # Convert to pixels and recenter
    print(fpX, fpY)
    cdMatrix = ref_wcs.getCdMatrix()
    # now rotate the CD matrix into the right orientation.
    nRot = img_detector.getOrientation().getNQuarter()
    rot = ((90.0 * nRot) % 360.0) * np.pi / 180.0
    rot_matrix = np.array([[np.cos(rot), np.sin(rot)],
                                [-np.sin(rot), np.cos(rot)]])
    cdMatrix_rot = np.dot(rot_matrix, cdMatrix)
    pixelOrigin = Point2D(img_bbox.centerX, img_bbox.centerY)
    skyOrigin = ref_wcs.pixelToSky(Point2D(fpX, fpY))
    img_wcs = afwGeom.makeSkyWcs(pixelOrigin, skyOrigin, cdMatrix_rot)
    return img_wcs


In [ ]:
def QueryGAIA(wcs, bbox, ra_center, dec_center, magLimit=17.0):
    radius = 800.0#max(bbox.getDimensions()) / 2.0 * np.sqrt(2.0) * 0.2
    radius = radius * u.arcsec # Enough to cover the bbox    
    cols = ['ra', 'dec', 'phot_g_mean_mag', 'designation']
    # Construct the ADQL query
    query = f"""
    SELECT *
    FROM gaiadr3.gaia_source
    WHERE CONTAINS(POINT({ra_center}, {dec_center}), CIRCLE(ra, dec, {radius.to(u.degree).value})) = 1
    AND phot_g_mean_mag < {magLimit} 
    """
    print(query)
    # Launch the query
    job = Gaia.launch_job_async(query)
    results = job.get_results()
    gaia_table = results.to_pandas()
    gaia_table = gaia_table[cols]
    # using the wcs to locate the stars inside the CCD minus npixedge
    ccdx,ccdy = wcs.skyToPixelArray(gaia_table['ra'],gaia_table['dec'], degrees=True)
    inCCD = bbox.contains(ccdx,ccdy)
    gaia_table['ccdx'] = ccdx
    gaia_table['ccdy'] = ccdy
    gaia_table['inCCD'] = inCCD
    return gaia_table


In [ ]:
def CheckWCSScience(camera, wcs, expId, detector, magLimit):
    detName = detector.getName()
    bbox = detector.getBBox()
    calexp = butler.get('preliminary_visit_image', detector=detector.getId(), visit=expId)
    skyCenter = wcs.pixelToSky(Point2D(bbox.centerX, bbox.centerY))
    ra_center = skyCenter.getRa().asDegrees()
    dec_center = skyCenter.getDec().asDegrees()
    gaia_table = QueryGAIA(wcs, bbox, ra_center, dec_center, magLimit=magLimit)
    #for i in range(len(gaia_table)):
    #    print(gaia_table['designation'][i], ccdx[i], ccdy[i], inCCD[i], gaia_table['phot_g_mean_mag'][i])
    xs = []
    ys = []
    mags = []
    for i in range(len(gaia_table)):
        x = gaia_table['ccdx'][i]
        y = gaia_table['ccdy'][i]
        if gaia_table['inCCD'][i]:
            xs.append(x)
            ys.append(y)
            mags.append(f"{gaia_table['phot_g_mean_mag'][i]:.1f}")
    
    fig = plt.figure(figsize=(16,16))
    
    myPlot = plot(calexp, stretch='ccs', figure=fig)
    ax = myPlot.get_axes()[0]
    ax.scatter(xs, ys\
                ,facecolors='none', edgecolors='magenta', s=200, lw=2)
    ax.set_title(f"{expId} detector {detector.getId()}")
    return fig


In [ ]:
"""
ampDict = {'Segment00':'C00', 'Segment01':'C01', 'Segment02':'C02', 'Segment03':'C03', 
           'Segment04':'C04', 'Segment05':'C05', 'Segment06':'C06', 'Segment07':'C07', 
           'Segment08':'C17', 'Segment09':'C16', 'Segment10':'C15', 'Segment11':'C14', 
           'Segment12':'C13', 'Segment13':'C12', 'Segment14':'C11', 'Segment15':'C10', }
"""

ampDict = {'Segment00':'C00', 'Segment01':'C01', 'Segment02':'C02', 'Segment03':'C03', 
           'Segment04':'C04', 'Segment05':'C05', 'Segment06':'C06', 'Segment07':'C07', 
           'Segment17':'C17', 'Segment16':'C16', 'Segment15':'C15', 'Segment14':'C14', 
           'Segment13':'C13', 'Segment12':'C12', 'Segment11':'C11', 'Segment10':'C10', }


def CheckWCSGuider(camera, wcs, expId, detector, magLimit, stampOnly=True):
    detName = detector.getName()
    if expId > 2025050900000:
        detName = detector.getName()
    else:
        print("Swapping SG0, SG1")
        # This is necessary because SG0 and SG1 were swapped for a time
        if detName.split('_')[1] == 'SG0':
            detName = detName.split('_')[0] + '_SG1'
        else:
            detName = detName.split('_')[0] + '_SG0'
    detector = camera[detName]
    bbox = detector.getBBox()
    lct = LsstCameraTransforms(camera,detName)
    background = 30
    dummy = np.zeros([bbox.endY, bbox.endX]) + background
    raw = butler.get("guider_raw", exposure=expId, detector=detector.getId(), instrument='LSSTCam')
    arr = raw[2].stamp_im.image.array
    arr = arr - np.median(arr)
    mdata = raw.metadata
    cols = mdata['ROICOLS']
    rows = mdata['ROIROWS']
    llX = mdata['ROICOL']
    llY = mdata['ROIROW']
    urX = llX + mdata['ROICOLS']
    urY = llY + mdata['ROIROWS']
    stamp_ampName = ampDict[mdata['ROISEG']]
    x0, y0 = lct.ampPixelToCcdPixel(llX, llY, stamp_ampName, detName)
    x1, y1 = lct.ampPixelToCcdPixel(urX, urY, stamp_ampName, detName)
    print(x0, x1, y0, y1)
    x0 = int(x0)
    y0 = int(y0)
    x1 = int(x1)
    y1 = int(y1)
    if x1 < x0:
        x0, x1 = x1, x0
        arr = np.fliplr(arr)
    if y1 < y0:
        y0, y1 = y1, y0
        arr = np.flipud(arr)
    print(x0, x1, y0, y1)
    bbox = Box2I(Point2I(x0,y0), Point2I(x1,y1))
    skyCenter = wcs.pixelToSky(Point2D(bbox.centerX, bbox.centerY))
    ra_center = skyCenter.getRa().asDegrees()
    dec_center = skyCenter.getDec().asDegrees()
    gaia_table = QueryGAIA(wcs, bbox, ra_center, dec_center, magLimit=magLimit)
    #print(len(gaia_table))
    #print(gaia_table)
    #for i in range(len(gaia_table)):
    #    print(gaia_table['designation'][i], gaia_table[ccdx][i], gaia_table[ccdy][i], inCCD[i], gaia_table['phot_g_mean_mag'][i])
    xs = []
    ys = []
    mags = []
    fudgeX, fudgeY = 0,0#guider_ccd_offsets(img_detName)
    for i in range(len(gaia_table)):
        x = gaia_table['ccdx'][i] + fudgeX
        y = gaia_table['ccdy'][i] + fudgeY
        if gaia_table['inCCD'][i]:
            xs.append(x)
            ys.append(y)
            print(gaia_table['designation'][i], x, y, gaia_table['phot_g_mean_mag'][i])
            mags.append(f"{gaia_table['phot_g_mean_mag'][i]:.1f}")

    dummy[y0:y1, x0:x1] = arr
    fig = plt.figure(figsize=(16,16))
    
    myPlot = plot(dummy, stretch='ccs', figure=fig)
    ax = myPlot.get_axes()[0]
    if stampOnly:
        ax.set_xlim(x0, x1)
        ax.set_ylim(y0, y1)
    ax.scatter(xs, ys\
                ,facecolors='none', edgecolors='magenta', s=2000, lw=4)
    ax.set_title(f"{expId} detector {detector.getId()} {stamp_ampName}")
    return fig #, raw[2].stamp_im


def guider_ccd_offsets(img_detName):
    if img_detName == 'R44_SG0':
        fudgeX = 30; fudgeY = -12
    elif img_detName == 'R44_SG1':
        fudgeX = -2; fudgeY = -18
    elif img_detName == 'R40_SG0':
        fudgeX = -5; fudgeY = -39
    elif img_detName == 'R40_SG1':
        fudgeX = -15; fudgeY = 34
    elif img_detName == 'R04_SG0':
        fudgeX = 45; fudgeY = 17
    elif img_detName == 'R04_SG1':
        fudgeX = 15; fudgeY = -30
    elif img_detName == 'R00_SG0':
        fudgeX = 7; fudgeY = -5
    elif img_detName == 'R00_SG1':
        fudgeX = 8; fudgeY = 11
    return fudgeX, fudgeY


In [ ]:
butler = Butler('LSSTCam', collections=["LSSTCam/raw/all", "LSSTCam/calib", 
                                        "LSSTCam/runs/quickLook", "LSSTCam/raw/guider"])
instrument = 'LSSTCam'

In [ ]:
expId = 2025112700370

ref_detName = 'R22_S11'
camera = LsstCam.getCamera()
ref_detector = camera[ref_detName]
calexp = butler.get('preliminary_visit_image', detector=ref_detector.getId(), visit=expId)
ref_wcs = calexp.getWcs()
magLimit = 13.0

fig = CheckWCSScience(camera, ref_wcs, expId, ref_detector, magLimit)

In [ ]:
expId = 2025071700032

ref_detName = 'R22_S11'
camera = LsstCam.getCamera()
ref_detector = camera[ref_detName]
calexp = butler.get('preliminary_visit_image', detector=ref_detector.getId(), visit=expId)
ref_wcs = calexp.getWcs()

img_detName = 'R33_S22'
img_detector = camera[img_detName]
img_wcs = MakeShiftedWCS(camera, ref_detector, ref_wcs, img_detector)
fig = CheckWCSScience(camera, img_wcs, expId, img_detector, magLimit)

In [ ]:
expId = 2025112700370

ref_detName = 'R22_S11'
camera = LsstCam.getCamera()
ref_detector = camera[ref_detName]
calexp = butler.get('preliminary_visit_image', detector=ref_detector.getId(), visit=expId)
ref_wcs = calexp.getWcs()

dets = ['R00_SG0', 'R00_SG1', 'R04_SG0', 'R04_SG1', 'R40_SG0', 'R40_SG1', 'R44_SG0', 'R44_SG1']
img_detName = dets[3]
img_detector = camera[img_detName]
img_wcs = MakeShiftedWCS(camera, ref_detector, ref_wcs, img_detector)
fig = CheckWCSGuider(camera, img_wcs, expId, img_detector, magLimit=14.0, stampOnly=False)

In [ ]:
# This all worked out.
expId = 2025071700031

def guider_ccd_offsets(img_detName):
    if img_detName == 'R44_SG0':
        fudgeX = 30; fudgeY = -12
    elif img_detName == 'R44_SG1':
        fudgeX = -2; fudgeY = -15
    elif img_detName == 'R40_SG0':
        fudgeX = -5; fudgeY = -35
    elif img_detName == 'R40_SG1':
        fudgeX = -15; fudgeY = 34
    elif img_detName == 'R04_SG0':
        fudgeX = 45; fudgeY = 17
    elif img_detName == 'R04_SG1':
        fudgeX = 15; fudgeY = -30
    elif img_detName == 'R00_SG0':
        fudgeX = 7; fudgeY = -5
    elif img_detName == 'R00_SG1':
        fudgeX = 8; fudgeY = 11
    return fudgeX, fudgeY


## The stuff below is thoughts on findning sources to align with GAIA sources.

In [ ]:
arr

In [ ]:
from lsst.afw.image import ImageF, MaskedImageF, ExposureF
#im = ImageF(arr)
#mim = MaskedImageF(im)
exp = ExposureF(mim)

In [ ]:
myPlot = plot(exp, stretch='ccs')
ax = myPlot.get_axes()[0]
ax.scatter(xs, ys\
            ,facecolors='none', edgecolors='magenta', s=2000, lw=4)


In [ ]:
charResult = runCharacterizeImage(exp, 20, 50)

In [ ]:
sourceCatalog = charResult.sourceCat
sources = sourceCatalog.asAstropy()
sources.keep_columns(['base_SdssCentroid_x', 'base_SdssCentroid_y', 'base_CircularApertureFlux_3_0_instFlux'])
print(len(sources))
xs = []
ys = []
for source in sources:
    x = source['base_SdssCentroid_x']
    y = source['base_SdssCentroid_y']
    print(x, y)
    xs.append(x)
    ys.append(y)


In [ ]:
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig

def runCharacterizeImage(exp, thresh, minPix):
    """Run the image characterization task, finding only bright sources.

    Parameters
    ----------
    exp : `lsst.afw.image.Exposure`
        The exposure to characterize.
    snr : `float`
        The SNR threshold for detection.
        Here we have modified it to be a numeric threshold
    minPix : `int`
        The minimum number of pixels to count as a source.

    Returns
    -------
    result : `lsst.pipe.base.Struct`
        The result from the image characterization task.
    """
    
    charConfig = CharacterizeImageConfig()
    charConfig.doMeasurePsf = False
    charConfig.doApCorr = True
    charConfig.doDeblend = True
    charConfig.doNormalizedCalibration = False
    charConfig.repair.doCosmicRay = False

    charConfig.detection.minPixels = minPix
    charConfig.detection.thresholdValue = thresh
    charConfig.detection.thresholdType = 'value'
    charConfig.detection.includeThresholdMultiplier = 1
    charConfig.detection.nSigmaToGrow = 0

    charConfig.psfIterations = 1
    charConfig.installSimplePsf.fwhm = 5
    charConfig.installSimplePsf.width = 51
    
    # fit background with the most simple thing possible as we don't need
    # much sophistication here. weighting=False is *required* for very
    # large binSizes.
    charConfig.background.approxOrderX = 1
    charConfig.background.approxOrderY = -1
    charConfig.background.binSize = max(exp.getWidth(), exp.getHeight())
    charConfig.background.weighting = False

    # set this to use all the same minimal settings as those above
    charConfig.detection.background = charConfig.background

    charTask = CharacterizeImageTask(config=charConfig)

    charResult = charTask.run(exp)
    return charResult


In [ ]:
query = "SELECT * FROM gaiadr3.gaia_source WHERE CONTAINS(POINT(94.79122305581761, -54.28368946268521), CIRCLE(ra, dec, 0.25)) = 1 AND phot_g_mean_mag < 14.0 "

In [ ]:
job = Gaia.launch_job_async(query)
results = job.get_results()
gaia_table = results.to_pandas()
len(results)